In [1]:
import os
from pathlib import Path

project_root = Path.cwd().parent.parent
os.chdir(project_root)
print(os.getcwd())

/homes/dwiersma/Desktop/internship


# Using the best model

In [2]:
import pickle as pkl
import tomllib

from src.ml.cforest import Cforest
from src.data import Data

# load config
with open("config.toml", "rb") as file:
    config = tomllib.load(file)

# load the microarray data (set in config.toml), not important here though
data = Data(config)

data.replace_sample_sep(".")

ma_data = data.get_mm_with_tt()
ma_data.columns = ma_data.columns.str.replace(" ", "_")
print(f"ma_data shape: {ma_data.shape}")

2024-01-16 12:35:55,757:src.log_manager:INFO:Loading data...
2024-01-16 12:36:46,037:src.log_manager:INFO:Data loaded in 50.2768 seconds


ma_data shape: (15403, 9710)


In [3]:
import pandas as pd

ma_data = pd.read_csv("data/subsets_sorted/ma_15ct_min100s.csv")

ma_data = ma_data.set_index("samples")

ma_data.columns = ma_data.columns.str.replace(" ", "_")

ma_data

response  \
samples                                         
GSM1001610_1025A.CEL      Lung adenocarcinoma   
GSM1001611_104A.CEL       Lung adenocarcinoma   
GSM1001612_1065A.CEL      Lung adenocarcinoma   
GSM1001613_1176B.CEL      Lung adenocarcinoma   
GSM1001614_1179A.CEL      Lung adenocarcinoma   
...                                       ...   
GSM97841.CEL               Lower grade glioma   
GSM97854.CEL               Lower grade glioma   
GSM97864.CEL               Lower grade glioma   
GSM97924.CEL          Glioblastoma multiforme   
GSM97929.CEL               Lower grade glioma   

                      consensus_independent_component_1  \
samples                                                   
GSM1001610_1025A.CEL                           0.495375   
GSM1001611_104A.CEL                            0.799637   
GSM1001612_1065A.CEL                           0.366373   
GSM1001613_1176B.CEL                           1.411042   
GSM1001614_1179A.CEL                          -0.856902   
...                                                 ...   
GSM97841.CEL                                  -1.779873   
GSM97854.CEL                                  -1.386407   
GSM97864.CEL                                  -1.538455   
GSM97924.CEL                                  -1.464206   
GSM97929.CEL                                  -1.454969   

                      consensus_independent_component_2  \
samples                                                   
GSM1001610_1025A.CEL                          -0.144994   
GSM1001611_104A.CEL                           -0.921576   
GSM1001612_1065A.CEL                          -0.482790   
GSM1001613_1176B.CEL                          -0.726494   
GSM1001614_1179A.CEL                          -0.739656   
...                                                 ...   
GSM97841.CEL                                  -0.303522   
GSM97854.CEL                                  -0.257295   
GSM97864.CEL                                  -0.269964   
GSM97924.CEL                                  -0.111741   
GSM97929.CEL                                  -0.102552   

                      consensus_independent_component_3  \
samples                                                   
GSM1001610_1025A.CEL                           0.653698   
GSM1001611_104A.CEL                            0.656700   
GSM1001612_1065A.CEL                           0.621760   
GSM1001613_1176B.CEL                          -0.918696   
GSM1001614_1179A.CEL                          -0.404778   
...                                                 ...   
GSM97841.CEL                                   0.138593   
GSM97854.CEL                                  -0.022165   
GSM97864.CEL                                  -0.057362   
GSM97924.CEL                                   0.198785   
GSM97929.CEL                                  -0.264292   

                      consensus_independent_component_4  \
samples                                                   
GSM1001610_1025A.CEL                           0.152382   
GSM1001611_104A.CEL                            1.470572   
GSM1001612_1065A.CEL                           1.370997   
GSM1001613_1176B.CEL                           0.294711   
GSM1001614_1179A.CEL                           0.858474   
...                                                 ...   
GSM97841.CEL                                  -0.469585   
GSM97854.CEL                                  -0.547859   
GSM97864.CEL                                  -0.592612   
GSM97924.CEL                                  -0.268494   
GSM97929.CEL                                  -0.758626   

                      consensus_independent_component_5  \
samples                                                   
GSM1001610_1025A.CEL                           0.974743   
GSM1001611_104A.CEL                            1.217788   
GSM1001612_1065A.CEL                           1.656947   
GSM1001613_1176B.CEL            

In [4]:
cforest = Cforest(data)

# tcga forest
with open("/students/2023-2024/master/dwiersma/subsets_sorted/20240111141017_9e445a/cforest_model.pkl", "rb") as file:
    fitted_model = pkl.load(file)

cforest.fitted_model = fitted_model

In [5]:
ypredict_probs = cforest.predict(newx=ma_data.drop(columns=["response"]), type="prob")
ypredict = cforest.predict(newx=ma_data.drop(columns=["response"]), type="response")
cforest.runID = "ma_assessment"
cforest.assess(ytrue=ma_data["response"], ypredict=ypredict, ypredict_probs=ypredict_probs, name="clustermap_ma")

2024-01-16 12:50:37,075:src.log_manager:INFO:Performance metrics
AUC-ROC =		0.9909591522794592
MCC =			0.8260298972679138
ARI =			0.7908827593456615
top-3 accuracy =	0.9751588129325874
2024-01-16 12:50:39,955:src.log_manager:INFO:Clustermap saved to /students/2023-2024/master/dwiersma/output/cforest/ma_assessment/clustermap_ma.png


Hmm that performance isn't very good.
Let's explore some more.

# Feature Importance

Feature importance may be calculated like below, but it may take a while. I shall therefore load it from a file.

In [ ]:
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

importr("partykit")
robjects.r.assign("best_forest", cforest.fitted_model)

varimp = pd.DataFrame(robjects.r("data.frame(varimp(best_forest, cores = 50, conditional = FALSE))"))

In [ ]:
varimp.T